In [15]:
%tensorflow_version 1.x

In [16]:
!cp '/content/drive/My Drive/tmp/glaucoma.zip' ./ #마운트 된 파일을 현재 디렉토리로 가져오는(./)

In [17]:
!unzip glaucoma.zip

Archive:  glaucoma.zip
 extracting: train/advanced/1.png    
 extracting: train/advanced/10.png   
 extracting: train/advanced/100.png  
 extracting: train/advanced/101.png  
 extracting: train/advanced/102.png  
 extracting: train/advanced/103.png  
  inflating: train/advanced/104.png  
 extracting: train/advanced/105.png  
 extracting: train/advanced/106.png  
 extracting: train/advanced/107.png  
 extracting: train/advanced/108.png  
 extracting: train/advanced/109.png  
 extracting: train/advanced/11.png   
 extracting: train/advanced/110.png  
 extracting: train/advanced/111.png  
 extracting: train/advanced/112.png  
 extracting: train/advanced/113.png  
 extracting: train/advanced/114.png  
 extracting: train/advanced/115.png  
 extracting: train/advanced/116.png  
  inflating: train/advanced/117.png  
 extracting: train/advanced/118.png  
 extracting: train/advanced/119.png  
 extracting: train/advanced/12.png   
 extracting: train/advanced/120.png  
  inflating: train/advanced

In [18]:
!ls -al #가져온 파일이 현재 디렉토리에 있는지 확인하려고..  없는것 같으니 디렉토리를 다시 만들어줌

total 121144
drwxr-xr-x 1 root root      4096 Jun 26 06:16 .
drwxr-xr-x 1 root root      4096 Jun 26 05:22 ..
drwxr-xr-x 1 root root      4096 Jun 19 16:15 .config
drwx------ 4 root root      4096 Jun 26 05:23 drive
drwxr-xr-x 4 root root      4096 Jun 26 05:28 glaucoma
-rw------- 1 root root 124016091 Jun 26 06:16 glaucoma.zip
drwxr-xr-x 1 root root      4096 Jun 17 16:18 sample_data
drwxr-xr-x 5 root root      4096 Jun 26 06:16 test
drwxr-xr-x 5 root root      4096 Jun 26 06:16 train


In [20]:
!mkdir glaucoma
!mv train glaucoma/
!mv test glaucoma/

mkdir: cannot create directory ‘glaucoma’: File exists
mv: cannot move 'train' to 'glaucoma/train': Directory not empty
mv: cannot move 'test' to 'glaucoma/test': Directory not empty


In [21]:
!ls -al

total 121144
drwxr-xr-x 1 root root      4096 Jun 26 06:16 .
drwxr-xr-x 1 root root      4096 Jun 26 05:22 ..
drwxr-xr-x 1 root root      4096 Jun 19 16:15 .config
drwx------ 4 root root      4096 Jun 26 05:23 drive
drwxr-xr-x 4 root root      4096 Jun 26 05:28 glaucoma
-rw------- 1 root root 124016091 Jun 26 06:16 glaucoma.zip
drwxr-xr-x 1 root root      4096 Jun 17 16:18 sample_data
drwxr-xr-x 5 root root      4096 Jun 26 06:16 test
drwxr-xr-x 5 root root      4096 Jun 26 06:16 train


3일차 VGG16_classification_and_cumtom_data_training.ipynb.. 코랩에서 오픈하고 '디렉토리 구조의 데이터의 학습 최소코드 부분 전체 copy해옴

In [24]:
import os

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import VGG16


conv_layers = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
conv_layers.trainable = False

model = models.Sequential()

model.add(conv_layers)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
#model.add(layers.Dense(10, activation='softmax'))
model.add(layers.Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop", metrics=['acc'])


data_aug_generator = ImageDataGenerator(
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      vertical_flip=False,
      preprocessing_function=vgg16.preprocess_input
)

data_no_aug_generator = ImageDataGenerator(
      preprocessing_function=vgg16.preprocess_input
)

train_data_generator = data_aug_generator.flow_from_directory(
      "glaucoma/train",
      target_size=(224,224),
      batch_size=64,
      class_mode='sparse'
)

test_data_generator = data_no_aug_generator.flow_from_directory(
      "glaucoma/test",
      target_size=(224,224),
      class_mode='sparse'
)


model.fit_generator(
      train_data_generator,
      validation_data=test_data_generator,
      validation_steps=5,
      steps_per_epoch=train_data_generator.samples/64,
      epochs=10
)


y_ = model.predict_generator(
      test_data_generator,
      steps=test_data_generator.samples/64
)

custom_labels = list(test_data_generator.class_indices.keys())
predicted = np.argmax(y_, axis=1)
print(predicted[0], custom_labels[predicted[0]])


Found 771 images belonging to 3 classes.
Found 773 images belonging to 3 classes.
Epoch 1/10
13/12 [================================] - 13s 964ms/step - loss: 63.3986 - acc: 0.5785 - val_loss: 6.5066 - val_acc: 0.5250
Epoch 2/10
13/12 [================================] - 11s 845ms/step - loss: 4.3969 - acc: 0.7121 - val_loss: 4.2571 - val_acc: 0.5813
Epoch 3/10
13/12 [================================] - 11s 849ms/step - loss: 1.4310 - acc: 0.7704 - val_loss: 1.6509 - val_acc: 0.5312
Epoch 4/10
13/12 [================================] - 11s 864ms/step - loss: 0.8261 - acc: 0.7860 - val_loss: 1.1702 - val_acc: 0.7063
Epoch 5/10
13/12 [================================] - 11s 863ms/step - loss: 0.6592 - acc: 0.7613 - val_loss: 0.9131 - val_acc: 0.6000
Epoch 6/10
13/12 [================================] - 11s 862ms/step - loss: 0.7549 - acc: 0.7691 - val_loss: 1.1010 - val_acc: 0.5813
Epoch 7/10
13/12 [================================] - 11s 881ms/step - loss: 0.7665 - acc: 0.7756 - val_los